In [3]:
import sys
sys.path.insert(0, '/home/tim/Classes/CS535/PlanarGrasping/')
# sys.path.insert(0, '/home/tim/Classes/CS535/PlanarGrasping/network/')

import pickle
import matplotlib.pyplot as plt

from network.model import net, data_loader
from network import utils
import numpy as np

from IPython import display
%matplotlib inline

import torch

import torch.optim as optim

In [6]:
json_path = '/home/tim/Classes/CS535/PlanarGrasping/network/experiments/sdf_cnn/params.json'
params = utils.Params(json_path)
params.cuda = torch.cuda.is_available()

data_dir = '/home/tim/Classes/CS535/PlanarGrasping/network/data'

# fetch dataloaders
dataloaders = data_loader.fetch_dataloader(
    ['train', 'val'],data_dir, params)
train_dl = dataloaders['train']
val_dl = dataloaders['val']

# Define the model and optimizer
model = net.Net(params).cuda()
optimizer = optim.Adam(model.parameters(), lr=params.learning_rate)

model.eval()

# reload weights from restore_file if specified
weight_file = '/home/tim/Classes/CS535/PlanarGrasping/network/experiments/sdf_cnn/best.pth.tar'
utils.load_checkpoint(weight_file, model, optimizer)
print()


In [18]:
for i, (train_batch, labels_batch) in enumerate(val_dl):

    scan_pts = train_batch[0][:-1,:].numpy()
    bounds = [0, 15, -5, 5]
    xs = np.linspace(*bounds[:2], 100)
    ys = np.linspace(*bounds[2:], 100)
    X, Y = np.meshgrid(xs, ys)
    pts = np.column_stack([
        X.ravel(),
        Y.ravel()
    ])

    dists = []
    for pt in pts:
        s = torch.Tensor(np.vstack([scan_pts, pt])).cuda()
        dists.append(model.forward(s).cpu().detach().numpy())

    fig, ax = plt.subplots()
    # ax.scatter(X.ravel(), Y.ravel(), c=dists)
    ax.tricontourf(X.ravel(), Y.ravel(), dists, levels=14, cmap="viridis")
    ax.tricontour(X.ravel(), Y.ravel(), dists, levels=[0.4])
    ax.plot(scan_pts[:,0], scan_pts[:,1])
    plt.pause(0.01)

ValueError: 'red' is not a valid value for name; supported values are 'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r', 'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r', 'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r', 'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r', 'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r', 'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r', 'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r', 'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r', 'Spectral', 'Spectral_r', 'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr', 'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r', 'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cividis', 'cividis_r', 'cool', 'cool_r', 'coolwarm', 'coolwarm_r'

In [49]:
for i, (train_batch, labels_batch) in enumerate(val_dl):
    scan_pts = train_batch[0][:-1,:].numpy()
    query_pt = train_batch[0][-1,:].numpy()
    print(query_pt)
    print(labels_batch[0])
    print(model.forward(train_batch[0].float().cuda()))
    break

[12.70627143  2.04551195]
tensor(2.6959, dtype=torch.float64)
tensor(0.6596, device='cuda:0', grad_fn=<SqueezeBackward0>)
